In [1]:
# clone yolov5 repo
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

Cloning into 'yolov5'...
remote: Enumerating objects: 15898, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 15898 (delta 60), reused 77 (delta 37), pack-reused 15768
Receiving objects: 100% (15898/15898), 14.69 MiB | 5.74 MiB/s, done.
Resolving deltas: 100% (10872/10872), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 5.3 MB/s eta 

In [3]:
from models.common import DetectMultiBackend
from utils.general import (LOGGER, TQDM_BAR_FORMAT, check_amp, check_dataset, check_file, check_git_info,
                           check_git_status, check_img_size, check_requirements, check_suffix, check_yaml, colorstr,
                           get_latest_run, increment_path, init_seeds, intersect_dicts, labels_to_class_weights,
                           labels_to_image_weights, methods, one_cycle, print_args, print_mutation, strip_optimizer,
                           yaml_save)
from utils.dataloaders import create_dataloader
from utils.loss import ComputeLoss
from utils.torch_utils import select_device, smart_inference_mode
from utils.metrics import bbox_iou

from tqdm import tqdm
import torch
import yaml
IMAGE_SCALE = 2.0 / 255 # TODO: may need to tune this
import roboflow
from utils.torch_utils import (EarlyStopping, ModelEMA, de_parallel, select_device, smart_DDP, smart_optimizer,
                               smart_resume, torch_distributed_zero_first)


# Dataset


In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="gMTVjr2tVu8feJmkrnPs")
project = rf.workspace("aps360-project-6tmc3").project("aps360_-2")
dataset = project.version(7).download("yolov5")


loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to APS360_(2)-7 in yolov5pytorch:: 100%|██████████| 4583/4583 [00:01<00:00, 3273.09it/s]


# Attacker Module

In [5]:
# ref: https://github.com/amy-choi/AttackDefenseYOLO/blob/main/attacker.py
class FGSMAttacker:
    def __init__(self, epsilon, device, attack_type="total"):
        self.epsilon = epsilon * IMAGE_SCALE
        self.device = device
        self.attack_type = attack_type

    def attack(self, image_clean, label, yolo_loss, model):
        image_clean = image_clean.clone().detach().to(self.device)
        # generate an initial perturbation
        delta = (
            torch.zeros_like(image_clean)
            .uniform_(-self.epsilon, self.epsilon)
            .to(self.device)
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        delta.requires_grad = True

        pred = model(image_clean + delta)


        loss_total, _, loss_loc, loss_obj, loss_cls = yolo_loss(
            pred, label
        )

        if self.attack_type == "cls":
            loss = loss_cls
        elif self.attack_type == "loc":
            loss = loss_loc
        elif self.attack_type == "conf":
            loss = loss_obj
        elif self.attack_type == "total":
            loss = loss_total
        elif self.attack_type == "conf_loc":
            loss = loss_obj + loss_loc
        elif self.attack_type == "conf_cls":
            loss = loss_obj + loss_cls
        elif self.attack_type == "cls_loc":
            loss = loss_cls + loss_loc

        loss.backward()
        grad = delta.grad.detach()

        delta.data = torch.clamp(
            delta + self.epsilon * torch.sign(grad), -self.epsilon, self.epsilon
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        adv_img = (image_clean + delta).detach()

        return adv_img


    def generate_cls(self, image_clean, label, yolo_loss, model):
        image_clean = image_clean.clone().detach().to(self.device)  # .cuda()
        delta = (
            torch.zeros_like(image_clean)
            .uniform_(-self.epsilon, self.epsilon)
            .to(self.device)
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        delta.requires_grad = True

        outputs = model(image_clean + delta)
        losses = 0
        num_pos_all = 0

        for l in range(len(outputs)):
            _, _, _, loss_cls, num_pos = yolo_loss(l, outputs[l], label)
            losses += loss_cls
            num_pos_all += num_pos

        loss = losses / num_pos_all

        loss.backward()
        grad = delta.grad.detach()

        delta.data = torch.clamp(
            delta + self.epsilon * torch.sign(grad), -self.epsilon, self.epsilon
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        adv_img = (image_clean + delta).detach()

        return adv_img

    def generate_obj(self, image_clean, label, yolo_loss, model):
        image_clean = image_clean.clone().detach().to(self.device)  # .cuda()
        delta = (
            torch.zeros_like(image_clean)
            .uniform_(-self.epsilon, self.epsilon)
            .to(self.device)
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        delta.requires_grad = True

        outputs = model(image_clean + delta)
        losses = 0
        num_pos_all = 0

        for l in range(len(outputs)):
            _, _, loss_obj, _, num_pos = yolo_loss(l, outputs[l], label)
            losses += loss_obj
            num_pos_all += num_pos

        loss = losses / num_pos_all

        loss.backward()
        grad = delta.grad.detach()

        delta.data = torch.clamp(
            delta + self.epsilon * torch.sign(grad), -self.epsilon, self.epsilon
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        adv_img = (image_clean + delta).detach()

        return adv_img

    def generate_loc(self, image_clean, label, yolo_loss, model):
        image_clean = image_clean.clone().detach().to(self.device)  # .cuda()
        delta = (
            torch.zeros_like(image_clean).uniform_(-self.epsilon, self.epsilon).cuda()
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        delta.requires_grad = True

        outputs = model(image_clean + delta)
        losses = 0
        num_pos_all = 0

        for l in range(len(outputs)):
            _, loss_loc, _, _, num_pos = yolo_loss(l, outputs[l], label)
            losses += loss_loc
            num_pos_all += num_pos

        loss = losses / num_pos_all

        loss.backward()
        grad = delta.grad.detach()

        delta.data = torch.clamp(
            delta + self.epsilon * torch.sign(grad), -self.epsilon, self.epsilon
        )
        delta.data = torch.max(torch.min(1 - image_clean, delta.data), 0 - image_clean)
        adv_img = (image_clean + delta).detach()

        return adv_img


# Adapted Loss fn

In [6]:
class ComputeIndividualLoss(ComputeLoss):
  def __init__(self, model, autobalance=False):
    super().__init__(model, autobalance)

  # override the __call__ function to return the component loss tensors
  def __call__(self, p, targets):  # predictions, targets
    lcls = torch.zeros(1, device=self.device)  # class loss
    lbox = torch.zeros(1, device=self.device)  # box loss
    lobj = torch.zeros(1, device=self.device)  # object loss
    tcls, tbox, indices, anchors = self.build_targets(p, targets)  # targets

    # Losses
    for i, pi in enumerate(p):  # layer index, layer predictions
        b, a, gj, gi = indices[i]  # image, anchor, gridy, gridx
        tobj = torch.zeros(pi.shape[:4], dtype=pi.dtype, device=self.device)  # target obj

        n = b.shape[0]  # number of targets
        if n:
            # pxy, pwh, _, pcls = pi[b, a, gj, gi].tensor_split((2, 4, 5), dim=1)  # faster, requires torch 1.8.0
            pxy, pwh, _, pcls = pi[b, a, gj, gi].split((2, 2, 1, self.nc), 1)  # target-subset of predictions

            # Regression
            pxy = pxy.sigmoid() * 2 - 0.5
            pwh = (pwh.sigmoid() * 2) ** 2 * anchors[i]
            pbox = torch.cat((pxy, pwh), 1)  # predicted box
            iou = bbox_iou(pbox, tbox[i], CIoU=True).squeeze()  # iou(prediction, target)
            lbox += (1.0 - iou).mean()  # iou loss

            # Objectness
            iou = iou.detach().clamp(0).type(tobj.dtype)
            if self.sort_obj_iou:
              j = iou.argsort()
              b, a, gj, gi, iou = b[j], a[j], gj[j], gi[j], iou[j]
            if self.gr < 1:
                iou = (1.0 - self.gr) + self.gr * iou
            tobj[b, a, gj, gi] = iou  # iou ratio

            # Classification
            if self.nc > 1:  # cls loss (only if multiple classes)
                t = torch.full_like(pcls, self.cn, device=self.device)  # targets
                t[range(n), tcls[i]] = self.cp
                lcls += self.BCEcls(pcls, t)  # BCE

            # Append targets to text file
            # with open('targets.txt', 'a') as file:
            #     [file.write('%11.5g ' * 4 % tuple(x) + '\n') for x in torch.cat((txy[i], twh[i]), 1)]

        obji = self.BCEobj(pi[..., 4], tobj)
        lobj += obji * self.balance[i]  # obj loss
        if self.autobalance:
            self.balance[i] = self.balance[i] * 0.9999 + 0.0001 / obji.detach().item()

    if self.autobalance:
        self.balance = [x / self.balance[self.ssi] for x in self.balance]
    lbox *= self.hyp['box']
    lobj *= self.hyp['obj']
    lcls *= self.hyp['cls']
    bs = tobj.shape[0]  # batch size

    return (lbox + lobj + lcls), torch.cat((lbox, lobj, lcls)).detach(), lbox, lobj, lcls

# adv examples

In [7]:
from utils.torch_utils import select_device, smart_inference_mode
from models.yolo import Model
from utils.general import (LOGGER, TQDM_BAR_FORMAT, check_amp, check_dataset, check_file, check_git_info,
                           check_git_status, check_img_size, check_requirements, check_suffix, check_yaml, colorstr,
                           get_latest_run, increment_path, init_seeds, intersect_dicts, labels_to_class_weights,
                           labels_to_image_weights, methods, one_cycle, print_args, print_mutation, strip_optimizer,
                           yaml_save)
from utils.torch_utils import (EarlyStopping, ModelEMA, de_parallel, select_device, smart_DDP, smart_optimizer,
                               smart_resume, torch_distributed_zero_first)

from utils.loss import ComputeLoss
import torch
import os
import yaml
LOCAL_RANK = int(os.getenv('LOCAL_RANK', -1))  # https://pytorch.org/docs/stable/elastic/run.html
RANK = int(os.getenv('RANK', -1))
WORLD_SIZE = int(os.getenv('WORLD_SIZE', 1))
import numpy as np
import matplotlib.pyplot as plt


In [8]:
def _load_model(weights_path, device, nc, anchors):
  '''
    @param data_folder: path to the data.yaml file
    return a model that is loaded with the specified weights
  '''
  ckpt = torch.load(weights_path, map_location="cpu")
  model = Model(ckpt['model'].yaml, ch=3, nc=nc, anchors=anchors).to(device)  # create
  exclude = []
  # exclude = ['anchor'] if anchors else []
  csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
  csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
  model.load_state_dict(csd, strict=False)  # load
  print(f'Transferred {len(csd)}/{len(model.state_dict())} items from {weights_path}')  # report)
  return model

In [33]:
def get_adv_examples(epsilon,
                     weights_path,
                     num_batches,
                     att_type,
                     data_folder,
                     hyperparam_path,
                     batch_size=32,
                     imgsz=640,
                     device='',
                     seed=0,
                     workers=2,
                     label_smoothing=0.0):
  '''
      @param weights_path: path to weights of a model that you would like to generate
                            adversarial examples on
      @param data_folder: clean data
      @param att_type: # takes "cls", "loc", or "conf"

  '''
  single_cls = False
  # load in the hyperparameters
  if isinstance(hyperparam_path, str):
        with open(hyperparam_path, errors='ignore') as f:
            hyp = yaml.safe_load(f)  # load hyps dict

  cuda = device.type != 'cpu'
  init_seeds(seed + 1 + RANK, deterministic=True)

  # load data dict
  with torch_distributed_zero_first(LOCAL_RANK):
      data_dict = check_dataset(data_folder)  # check if None
  nc = int(data_dict["nc"])
  names = data_dict["names"]
  is_coco = False

  # load model with weights
  model = _load_model(weights_path, device, nc, hyp.get("anchors"))
  amp = check_amp(model)  # check AMP

  gs = max(int(model.stride.max()), 32)  # grid size (max stride)
  imgsz = check_img_size(imgsz, gs, floor=gs * 2)  # verify imgsz is gs-multiple

  ema = ModelEMA(model) if RANK in {-1, 0} else None

  train_path, val_path = data_dict['train'], data_dict['val']
  train_loader, dataset = create_dataloader(train_path,
                                            imgsz,
                                            batch_size // WORLD_SIZE,
                                            gs,
                                            single_cls,
                                            hyp=hyp,
                                            augment=False,
                                            cache=None,
                                            rect=False, #unless you specify this flag when training
                                            rank=LOCAL_RANK,
                                            workers=workers,
                                            image_weights=False, # unless you specify this flag when training
                                            quad=False, # unless you specify this flag when training
                                            prefix=colorstr('train: '),
                                            shuffle=True,
                                            seed=seed)
  labels = np.concatenate(dataset.labels, 0)
  mlc = int(labels[:, 0].max())  # max label class
  assert mlc < nc, f'Label class {mlc} exceeds nc={nc} in {data_folder}. Possible class labels are 0-{nc - 1}'

  # Model attributes
  nl = de_parallel(model).model[-1].nl  # number of detection layers (to scale hyps)
  hyp['box'] *= 3 / nl  # scale to layers
  hyp['cls'] *= nc / 80 * 3 / nl  # scale to classes and layers
  hyp['obj'] *= (imgsz / 640) ** 2 * 3 / nl  # scale to image size and layers
  hyp['label_smoothing'] = label_smoothing
  model.nc = nc  # attach number of classes to model
  model.hyp = hyp  # attach hyperparameters to model
  model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device) * nc  # attach class weights
  model.names = names

  # init YOLO loss
  compute_loss = ComputeIndividualLoss(model)

  attacker = FGSMAttacker(epsilon, device, att_type)

  model.train()

  pbar = enumerate(train_loader)
  pbar = tqdm(pbar, total=len(train_loader), bar_format=TQDM_BAR_FORMAT)  # progress bar
  # optimizer.zero_grad() ??????
  adv_images = []
  original_images = []
  adv_loss = 0.0
  for batch_i, (im, targets, paths, shapes) in (pbar):
      if batch_i == num_batches:
        break
      if cuda:
          im = im.to(device, non_blocking=True).float() / 255
          targets = targets.to(device)
      nb, _, height, width = im.shape  # batch size, channels, height, width

      adv_image = attacker.attack(im, targets, compute_loss, model)
      # evaluate model's loss
      with torch.no_grad():
          if cuda:
              adv_image = adv_image.to(device)
          adv_pred = model(adv_image)
          loss_total, _, _, _, _ = compute_loss(
              adv_pred, targets
          )
          adv_loss += loss_total.cpu().item()

      adv_images.append(adv_image)
      original_images.append(im)
  return torch.cat(adv_images), torch.cat(original_images), adv_loss / len(pbar)

In [34]:
adv_examples_cls, orig_examples_cls, mean_loss_cls = get_adv_examples(6.0, "/content/bestC.pt", 10, "cls", "/content/yolov5/APS360_(2)-7/data.yaml", 'data/hyps/hyp.scratch-low.yaml', device=torch.device("cuda:0"), batch_size=4)
adv_examples_obj, orig_examples_obj, mean_loss_obj = get_adv_examples(6.0, "/content/bestC.pt", 10, "conf", "/content/yolov5/APS360_(2)-7/data.yaml", 'data/hyps/hyp.scratch-low.yaml', device=torch.device("cuda:0"), batch_size=4)
adv_examples_loc, orig_examples_loc, mean_loss_loc = get_adv_examples(6.0, "/content/bestC.pt", 10, "loc", "/content/yolov5/APS360_(2)-7/data.yaml", 'data/hyps/hyp.scratch-low.yaml', device=torch.device("cuda:0"), batch_size=4)
adv_examples_t, orig_examples_t, mean_loss_t = get_adv_examples(3.0, "/content/bestC.pt", 1, "total", "/content/yolov5/APS360_(2)-7/data.yaml", 'data/hyps/hyp.scratch-low.yaml', device=torch.device("cuda:0"), batch_size=4)


                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.C3                        [384, 384, 6]                 
  7                -1  1   2655744  models.common.Conv                      [384, 768, 3, 2]              
  8                -1  2   4134912  

Transferred 481/481 items from /content/bestC.pt


train: Scanning /content/yolov5/APS360_(2)-7/train/labels.cache... 1766 images, 28 backgrounds, 0 corrupt: 100%|██████████| 1766/1766 [00:00<?, ?it/s]
  2%|▏         | 10/442 [00:03<02:27,  2.92it/s]

                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.

Transferred 481/481 items from /content/bestC.pt


train: Scanning /content/yolov5/APS360_(2)-7/train/labels.cache... 1766 images, 28 backgrounds, 0 corrupt: 100%|██████████| 1766/1766 [00:00<?, ?it/s]
  2%|▏         | 10/442 [00:03<02:30,  2.87it/s]

                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.

Transferred 481/481 items from /content/bestC.pt


train: Scanning /content/yolov5/APS360_(2)-7/train/labels.cache... 1766 images, 28 backgrounds, 0 corrupt: 100%|██████████| 1766/1766 [00:00<?, ?it/s]
  2%|▏         | 10/442 [00:03<02:26,  2.96it/s]

                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.

Transferred 481/481 items from /content/bestC.pt


train: Scanning /content/yolov5/APS360_(2)-7/train/labels.cache... 1766 images, 28 backgrounds, 0 corrupt: 100%|██████████| 1766/1766 [00:00<?, ?it/s]
  0%|          | 1/442 [00:00<03:02,  2.42it/s]


In [35]:
print(mean_loss_t)
print(mean_loss_obj)
print(mean_loss_loc)
print(mean_loss_cls)

0.000136911397066591
0.001612100834490487
0.0013993959873914719
0.001595092566506895


In [ ]:
# save them as png files
# !mkdir /content/adv_examples
!mkdir /content/adv_examples/try_3_obj
for i in range(len(adv_examples_obj)):
  plt.imshow(adv_examples_obj[i].detach().cpu().permute((1,2,0)))
  plt.savefig(f"/content/adv_examples/try_3_obj/example_{i}.png")

In [37]:
# save them as png files
# !mkdir /content/orig_examples
!mkdir /content/orig_examples/try_3_obj
for i in range(len(orig_examples_obj)):
  plt.imshow(orig_examples_obj[i].detach().cpu().permute((1,2,0)))
  plt.savefig(f"/content/orig_examples/try_3_obj/example_{i}.png")

In [38]:
!python detect.py --source /content/adv_examples/try_3_obj --weights /content/bestC.pt --name adv_example_run_4

detect: weights=['/content/bestC.pt'], source=/content/adv_examples/try_3_obj, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=adv_example_run_4, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-200-g05e4c05 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
image 1/40 /content/adv_examples/try_3_obj/example_0.png: 480x640 1 seatbelt, 42.8ms
image 2/40 /content/adv_examples/try_3_obj/example_1.png: 480x640 1 noSeatbelt, 21.4ms
image 3/40 /content/adv_examples/try_3_obj/example_10.png: 480x640 1 noSeatbelt, 21.3ms
image 4/40 /content/adv_examples/try_3_obj/example_11.png: 480x640 2 sea

In [39]:
!python detect.py --source /content/orig_examples/try_3_obj --weights /content/bestC.pt --name orig_example_run_4

detect: weights=['/content/bestC.pt'], source=/content/orig_examples/try_3_obj, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=orig_example_run_4, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-200-g05e4c05 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
image 1/40 /content/orig_examples/try_3_obj/example_0.png: 480x640 2 seatbelts, 46.1ms
image 2/40 /content/orig_examples/try_3_obj/example_1.png: 480x640 1 noSeatbelt, 21.4ms
image 3/40 /content/orig_examples/try_3_obj/example_10.png: 480x640 1 noSeatbelt, 21.3ms
image 4/40 /content/orig_examples/try_3_obj/example_11.png: 480x64

In [ ]:
import shutil

# Specify the folder path you want to delete
# folder_path = '/content/original_examples'  # Replace this with the actual folder path
folder_path = '/content/adv_examples'  # Replace this with the actual folder path

# Use shutil.rmtree() to recursively remove the folder and its contents
shutil.rmtree(folder_path)

In [41]:
# zip the adversarial examples to be saved for training
import shutil

folder_path = "/content/yolov5/runs/detect/adv_example_run_4"
zip_path = "/content/yolov5/runs/detect/adv_example_run_4/zipfile.zip"

shutil.make_archive(zip_path.split(".")[0], 'zip', folder_path)

RuntimeError: ignored